In [4]:
%gui qt5

import pandas as pd
from PyQt5.QtWidgets import QApplication
from skimage import io
import numpy as np

from napari import Window, Viewer

In [5]:
def get_coords(df):
    coords = []
    for row in df.iterrows():
        coords.append([row[1].col, row[1].row])
        
    # Expert annotations are indexed from 1
    coords = np.array(coords) - 1
        
    return coords

In [6]:
def get_coords_worker(df):
    coords = []
    for row in df.iterrows():
        coords.append([row[1].centroid_x, row[1].centroid_y])
        
    return np.array(coords)

In [16]:
# Load the csvs
expert_anno_df = pd.read_csv('./expert_coords/ISS_rnd0_ch3_z0.csv')
worker_anno_df = pd.read_csv('./total_consensus_coords/ISS_rnd0_ch3_z0.csv')
her2_anno_df = pd.read_csv('./starfish_coords/HER2.csv')
vim_anno_df = pd.read_csv('./starfish_coords/VIM.csv')

# Load the image
dots_image = io.imread('./images/ISS_rnd0_ch3_z0.png', name='dots')

In [17]:
expert_anno_coords = get_coords(expert_anno_df)
worker_anno_coords = get_coords_worker(worker_anno_df)
her2_anno_coords = get_coords_worker(her2_anno_df)
vim_anno_coords = get_coords_worker(vim_anno_df)

In [20]:
app = QApplication.instance() or QApplication([])

# initialize the viewer
viewer = Viewer()
window = Window(viewer, show=False)
viewer._window = window

In [21]:
# Add an image to the viewer
viewer.add_image(dots_image)

# Add the expert annotations to the viewer
size = [10] * len(expert_anno_coords)
viewer.add_markers(expert_anno_coords, size=size, symbol='ring', face_color='yellow', name="expert")

# Add the worker annotations to the viewer
size = [10] * len(worker_anno_coords)
viewer.add_markers(worker_anno_coords, size=size, symbol='ring', name="worker")

# Add the starfish annotations to the viewer
size = [10] * len(her2_anno_coords)
viewer.add_markers(her2_anno_coords, size=size, symbol='ring', face_color='red', name="HER2")
size = [10] * len(vim_anno_coords)
viewer.add_markers(vim_anno_coords, size=size, symbol='ring', face_color='blue', name="VIM")

window.show()